Задание

Цель: изучить применение методов оптимизации для решения задачи классификации
Описание задания:
В домашнем задании необходимо применить полученные знания в теории оптимизации и машинном обучении для реализации логистической регрессии.
Этапы работы:**

    1. Загрузите данные. Используйте датасет с ирисами. Его можно загрузить непосредственно из библиотеки Sklearn. В данных оставьте только 2 класса: Iris Versicolor, Iris Virginica.
    2. Самостоятельно реализуйте логистическую регрессию, без использования метода LogisticRegression из библиотеки. Можете использовать библиотеки pandas, numpy, math для реализации. Оформите в виде функции. *Оформите в виде класса с методами.
    3. Реализуйте метод градиентного спуска. Обучите логистическую регрессию этим методом. Выберете и посчитайте метрику качества. Метрика должна быть одинакова для всех пунктов домашнего задания. Для упрощения сравнения выберете только одну метрику.
    4. Повторите п. 3 для метода скользящего среднего (Root Mean Square Propagation, RMSProp).
    5. Повторите п. 3 для ускоренного по Нестерову метода адаптивной оценки моментов (Nesterov–accelerated Adaptive Moment Estimation, Nadam).
    6. Сравните значение метрик для реализованных методов оптимизации. Можно оформить в виде таблицы вида |метод|метрика|время работы| (время работы опционально). Напишите вывод.

Для лучшего понимания темы и упрощения реализации можете обратиться к статье.

Для получение зачета по этому домашнему заданию, минимально, должно быть реализовано обучение логистической регрессии и градиентный спуск.

In [1]:
# импорт библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
# загрузка датасета из sklearn
from sklearn.datasets import load_iris
iris = load_iris()

In [3]:
# преобразование датасета в pandas
# df = pd.DataFrame(data=np.c_[dataset['data', 'feature_names'], dataset['target']], columns= dataset['feature_names'] + ['target']) # короткий способ
df = pd.DataFrame(iris['data'], columns=iris['feature_names']) # загрузка данных
df['target'] = iris.target # загрузка таргетов
df['names'] = pd.Categorical.from_codes(iris.target, iris.target_names) # добавление названий
df = df[df['target'].isin([1,2])] # убираю setosa
df.target = df.target.replace(1, 0) # замена класса с 1 на 0
df.target = df.target.replace(2, 1) # замена класса с 2 на 1
df = df.drop(columns=['names']) # убираю названия
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
50,7.0,3.2,4.7,1.4,0
51,6.4,3.2,4.5,1.5,0
52,6.9,3.1,4.9,1.5,0
53,5.5,2.3,4.0,1.3,0
54,6.5,2.8,4.6,1.5,0


In [4]:
# подготовка данных
X = df.drop('target', axis = 1).values # данные
y = df.target.values # целевые значения

In [5]:
# X = np.c_[np.ones(len(X)), X] # тренировался с добавлением единичных позиций в вектор, но в результате не стал использовать, т.к. точность с ними оказалась ниже

In [6]:
# делю данные на тренировочную и тестовую выборки
# по стуи, в задании это не требуется, убираю этот шаг
# x_train, x_test, y_train, y_test = train_test_split(x ,y, test_size=0.3, random_state=42)

In [7]:
# функция сигмоиды

def sigmoid(z):    
    y_proba = 1 / (1+np.exp(-z))
    return y_proba

# то же в краткой записи
# sigmoid = lambda z: 1 / (1+np.exp(-z))

In [8]:
# визуализация сигмоиды
# делал для понимания
# z = np.linspace(-10, 10, 1000)
# plt.plot(z, sigmoid(z))
# plt.show()

In [9]:
# задаём изначальный вес и биас
# создаёт нулевую матрицу по количеству признаков
# def k_0(n_features):
#     w = np.zeros((1,n_features))
#     b = 0
#     return w,b

In [10]:
# проверяем на количестве признаков
# xt.shape[1]
# w, b = k_0(xt.shape[1])
# w, b
# # выдаёт нулевую матрицу из 4-х столбцов по количеству признаков

In [11]:
# вычисляю z для сигмы
# z = np.dot(xt.T, w)
# z

In [12]:
# создаю функцию потерь
def logloss(y, y_proba):
    logloss_1 = np.sum(np.log(y_proba[y == 1] + 1e-30))
    logloss_0 = np.sum(np.log(1 - y_proba[y == 0] + 1e-30))
    logloss_total = -(logloss_0 + logloss_1) / len(y)
    return logloss_total

In [13]:
# создаю функцию градиента
def grad_logloss(X, W, y):
    y_proba = sigmoid(X @ W)
    grad = X.T @ (y_proba - y)
    return grad

In [14]:
# нормализация данных, без этого не работало и показывало полную ерунду
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)
X.mean(axis=0), X.std(axis=0)

(array([ 3.28848060e-15, -3.63598041e-15, -5.42066392e-16, -1.28397293e-15]),
 array([1., 1., 1., 1.]))

In [15]:
# реализую логистическую регрессию
def log_reg(X, y, lr=0.001, iterations=1000, report_iter_frequency=50): # создаю функцию с указанными по умолчанию learning_rate и количеством итераций
    m, n = X.shape # строки и столбцы в выборке
    w = np.zeros(n) # задаю изначальный вес

    for i in range(iterations):
        grad = grad_logloss(X, w, y)
        w -= lr*(grad) # обновление веса по градиенту сдвиг веса по радиенту, при тренировка расписывал текущий и следующий вес, но в финальной реализации решил упростить
        
        # считаю предсказание и точность
        y_proba = sigmoid(np.dot(X, w))
        y_class = np.where(y_proba >= 0.5, 1, 0) # предсказание класса
        accuracy = (y_class == y).sum() / len(y) # считаю точность
        
        if i % report_iter_frequency == 0:
            print(f"iteration: {i}")
            print(f"weight: {w}")
            print(logloss(y, y_proba))
            print(f"accuracy: {accuracy}\n")
    return
log_reg(X, y, 0.001, 1000, 25)

iteration: 0
weight: [0.02471525 0.01540399 0.03932118 0.04140646]
0.6534814598101053
accuracy: 0.9

iteration: 25
weight: [0.26227998 0.10355101 0.57930078 0.64312087]
0.3402175815645784
accuracy: 0.93

iteration: 50
weight: [0.29241447 0.04711189 0.84848468 0.96245999]
0.26816231909758037
accuracy: 0.94

iteration: 75
weight: [ 0.27971422 -0.02706686  1.03944636  1.19232669]
0.23013375552561827
accuracy: 0.95

iteration: 100
weight: [ 0.2527156  -0.1002897   1.19096275  1.37495354]
0.20518728205118872
accuracy: 0.95

iteration: 125
weight: [ 0.22054254 -0.16810862  1.31806069  1.52750598]
0.18718107955459737
accuracy: 0.95

iteration: 150
weight: [ 0.18684912 -0.22965156  1.42834473  1.65896679]
0.17344976263131293
accuracy: 0.95

iteration: 175
weight: [ 0.15325465 -0.28514352  1.52626117  1.77472136]
0.16258448048313537
accuracy: 0.95

iteration: 200
weight: [ 0.1205017  -0.3351427   1.61465676  1.87828446]
0.15374983975236298
accuracy: 0.96

iteration: 225
weight: [ 0.08892042 -0.

In [16]:
# полный тренировочный набор логистической регрессии

# def log_reg(X, y, lr=0.001, iterations=1000): # создаю функцию с указанными по умолчанию learning_rate и количеством итераций
#     m, n = X.shape # строки и столбцы в выборке
#     #w = np.zeros(n)
    
#     np.random.seed(8)
#     w = np.random.randn(X.shape[1])
    
#     nw = w

#     b = 0

#     # next_w = w

#     for i in range(iterations):
#         grad = grad_logloss(X, w, y)
#         cw = nw
#         nw = cw - lr*grad
#         w -= lr*(grad) # обновление веса по градиенту сдвиг веса по радиенту
        
#         # считаю предсказание и точность
#         y_proba = sigmoid(np.dot(X, w))
#         y_class = np.where(y_proba >= 0.5, 1, 0) # предсказание класса
#         accuracy = (y_class == y).sum() / len(y)
        
#         if i % 10 == 0:
#             print(f"{cw} | {nw}")
#             print(accuracy)
#     return

        # cur_w = next_w
        # grad = grad_logloss(X, y, w) / m # реализация градиента через функциию
        # next_w = cur_w - lr*grad # движение по градиенту вниз
        # y_proba = sigmoid(np.dot(X, next_w))
        # y_class = np.where(y_proba >= 0.5, 1, 0)

        # print(y_proba)
        
        # if i % 5 == 0: # проверка каждые 5 итераций
        #     print(f"Итерация: {i}")
        #     y_proba = sigmoid(np.dot(x, next_w))
        #     y_class = np.where(y_proba >= 0.5, 2, 1)
        #     accuracy = (y_class == y).sum() / len(y)
        #     # print(f"Logloss {logloss(y, y_proba)}")
        #     print(f"{y_proba}")
        #     print(f"Accuracy {accuracy}")
        #     print(f'\n') 


        # реализация градиента построчно
        # z = np.dot(x, w) + b # рассчитывает аргумент для сигмоиды
        # y_proba = sigmoid(z) # рассчитывает y_proba
        # grad_z = y_proba - y # рассчитывает сдвиг y относительно реального y
        # grad = np.dot(x.T, grad_z) #/ m  # рассчитывает градиент
        # w -= lr*(grad) # сдвиг веса по радиенту
        # b -= lr*np.sum((y_proba - y) /m) # сдвиг биаса
        # много всяких print для тренировки
        # print(x.shape)
        # print(z)
        # print(y_proba)
        # print("weight", w)
        # print("cur_w", cur_w)
        # print("next_w", next_w)
        # print(y_class)
        # print("bias", b)
        # print(grad)
        # print(grad1)
        # print(m)
    # return

In [17]:
# logistic_regression(X, y, alpha=0.01, num_iterations=10)

In [18]:
# из ноутбука с доп. материалами

# eps = 0.001

# # первоначальное точка
# np.random.seed(8)
# W = np.random.randn(X.shape[1])

# # размер шага (learning rate)
# learning_rate = 0.001

# next_W = W

# # количество итерация 
# n = 100
# for i in range(n):
#     cur_W = next_W

#     # движение в негативную сторону вычисляемого градиента
#     next_W = cur_W - learning_rate * grad_logloss(X, W, y)

#     # остановка когда достигнута необходимая степень точности
#     if np.linalg.norm(cur_W - next_W) <= eps:
#         break

#     if i % 10 == 0:
#         # print(f"Итерация: {i}")
#         # print(f"Текущая точка {cur_W}| Следующая точка {next_W}")
#         print(f"{cur_W} | {next_W}")
#         y_proba = sigmoid(X @ next_W)
#         y_class = np.where(y_proba >= 0.5, 1, 0)
#         # print(y_class)
#         accuracy = (y_class == y).sum() / len(y)
#         # print(f"Logloss {logloss(y, y_proba)}")
#         # print(grad_logloss(X, W, y))
#         print(f"Accuracy {accuracy}")
#         print("--------------------------------------------------------") 

#         # visualize(next_W)   

Ссылка на полезную статью в kaggle https://www.kaggle.com/code/sagira/logistic-regression-math-behind-without-sklearn/notebook